In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image as pil_image
import imageio
import tensorflow
import sklearn.metrics
import itertools
%matplotlib inline
#plt.rcParams.update({'font.size': 32})
%config Completer.use_jedi = False

#import tensorflow.config.experimental
#from tensorflow.config.experimental import list_physical_devices, set_memory_growth
physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
def print_params(model):
    total_params = 0 # initialize counter for total params
    trainable_params = 0 # initialize counter for trainable params
    print('Layer Name\t\tType\t\tFilter shape\t\t# Parameters\tTrainable') # print column headings
    for layer in model.layers: # loop over layers
        lname = layer.name # grab layer name
        ltype = type(layer).__name__ # grab layer type
        ltype[ltype.find('/'):] # parse for only the last part of the string
        if ltype=='Conv2D': # print for convolutional layers
            weights = layer.get_weights()
            print(lname+'\t\t'+ltype+'\t\t'+str(weights[0].shape)+'\t\t'+\
                  str(layer.count_params())+'\t'+str(layer.trainable))
            if layer.trainable:
                trainable_params += layer.count_params()
            total_params += layer.count_params() # update number of params
        elif ltype=='MaxPooling2D': # print for max pool layers
            weights = layer.get_weights()
            print(lname+'\t\t'+ltype+'\t---------------\t\t---')
        elif ltype=='Flatten': # print for flatten layers
            print(lname+'\t\t'+ltype+'\t\t---------------\t\t---')
        elif ltype=='Dense': # print for dense layers
            weights = layer.get_weights()
            print(lname+'\t\t\t'+ltype+'\t\t'+str(weights[0].shape)+'\t\t'+\
                  str(layer.count_params())+'\t'+str(layer.trainable))
            if layer.trainable:
                trainable_params += layer.count_params()
            total_params += layer.count_params() # update number of params
        elif ltype=='Dropout': # print for dropout layers
            print(lname+'\t\t'+ltype+'\t\t------------------\t---')
    print('---------------')
    print('Total trainable parameters: '+str(trainable_params)) # print total params
    print('Total untrainable parameters: '+str(total_params-trainable_params))
    print('Total parameters: '+str(total_params))

In [4]:
def print_shapes(model):
    print('Layer Name\t\tType\t\tInput Shape\t\tOutput Shape\tTrainable')# print column headings
    for layer in model.layers:  # loop over layers
        lname = layer.name # grab layer name
        ltype = type(layer).__name__ # grab layer type
        ltype[ltype.find('/'):] # parse for only the last part of the string
        if ltype=='Conv2D': # print for convolutional layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape)+'\t'+str(layer.trainable))
        elif ltype=='MaxPooling2D': # print for maxpool layers
            print(lname+'\t\t'+ltype+'\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))
        elif ltype=='Flatten': # print for flatten layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))
        elif ltype=='Dense': # print for dense layers
            print(lname+'\t\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t\t'+\
                  str(layer.output_shape)+'\t'+str(layer.trainable))
        elif ltype=='Dropout': # print for dropout layers
            print(lname+'\t\t'+ltype+'\t\t'+str(layer.input_shape)+'\t'+\
                  str(layer.output_shape))

In [5]:
from keras.applications.vgg16 import preprocess_input
from astropy.io import fits
import skimage.transform

In [6]:
import keras
from keras.layers import Input
import keras.applications
input_tensor = Input(shape=(224,224,3))

In [7]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Conv2D, ReLU, concatenate, MaxPool2D, AvgPool2D

In [8]:
def fire_module(x,s1,e1,e3):
    s1x = Conv2D(s1,kernel_size = 1, padding = 'same')(x)
    s1x = ReLU()(s1x)
    e1x = Conv2D(e1,kernel_size = 1, padding = 'same')(s1x)
    e3x = Conv2D(e3,kernel_size = 3, padding = 'same')(s1x)
    x = concatenate([e1x,e3x])
    x = ReLU()(x)
    return x

In [9]:
import h5py
from keras.models import Model
from keras.layers import Input, Activation, Concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D

def SqueezeNet(nb_classes, inputs=(3, 222, 222)):
    """ Keras Implementation of SqueezeNet(arXiv 1602.07360)
    @param nb_classes: total number of final categories
    Arguments:
    inputs -- shape of the input images (channel, cols, rows)
    """

    input_img = Input(shape=inputs)
    conv1 = Convolution2D(
        96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
        strides=(2, 2), padding='same', name='conv1',
        data_format="channels_first")(input_img)
    maxpool1 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool1',
        data_format="channels_first")(conv1)
    fire2_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze',
        data_format="channels_first")(maxpool1)
    fire2_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1',
        data_format="channels_first")(fire2_squeeze)
    fire2_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2',
        data_format="channels_first")(fire2_squeeze)
    merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

    fire3_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_squeeze',
        data_format="channels_first")(merge2)
    fire3_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand1',
        data_format="channels_first")(fire3_squeeze)
    fire3_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand2',
        data_format="channels_first")(fire3_squeeze)
    merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

    fire4_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_squeeze',
        data_format="channels_first")(merge3)
    fire4_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand1',
        data_format="channels_first")(fire4_squeeze)
    fire4_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand2',
        data_format="channels_first")(fire4_squeeze)
    merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
    maxpool4 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool4',
        data_format="channels_first")(merge4)

    fire5_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_squeeze',
        data_format="channels_first")(maxpool4)
    fire5_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand1',
        data_format="channels_first")(fire5_squeeze)
    fire5_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand2',
        data_format="channels_first")(fire5_squeeze)
    merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

    fire6_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_squeeze',
        data_format="channels_first")(merge5)
    fire6_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand1',
        data_format="channels_first")(fire6_squeeze)
    fire6_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand2',
        data_format="channels_first")(fire6_squeeze)
    merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

    fire7_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_squeeze',
        data_format="channels_first")(merge6)
    fire7_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand1',
        data_format="channels_first")(fire7_squeeze)
    fire7_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand2',
        data_format="channels_first")(fire7_squeeze)
    merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

    fire8_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_squeeze',
        data_format="channels_first")(merge7)
    fire8_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand1',
        data_format="channels_first")(fire8_squeeze)
    fire8_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand2',
        data_format="channels_first")(fire8_squeeze)
    merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

    maxpool8 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool8',
        data_format="channels_first")(merge8)
    fire9_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_squeeze',
        data_format="channels_first")(maxpool8)
    fire9_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand1',
        data_format="channels_first")(fire9_squeeze)
    fire9_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand2',
        data_format="channels_first")(fire9_squeeze)
    merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

    fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
    conv10 = Convolution2D(
        nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='valid', name='conv10',
        data_format="channels_first")(fire9_dropout)

    global_avgpool10 = GlobalAveragePooling2D(data_format='channels_first')(conv10)
    softmax = Activation("softmax", name='softmax')(global_avgpool10)

    return Model(inputs=input_img, outputs=softmax)

In [10]:
train_df = pd.read_csv('train_data.csv',dtype=str)
val_df = pd.read_csv('val_data.csv',dtype=str)
test_df = pd.read_csv('test_data.csv',dtype=str)

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,\
                                                    directory='',\
                                                    xcol='filename',y_col='class',\
                                                    target_size=(222,222), color_mode='rgb',\
                                                    batch_size=128, class_mode='categorical',\
                                                    shuffle=True) #RETURN TO TRUEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_datagen.flow_from_dataframe(dataframe=val_df,\
                                                directory = '',\
                                                xcol='filename',ycol='class',\
                                                target_size=(222,222), color_mode='rgb',\
                                                batch_size=128, class_mode='categorical',\
                                                shuffle=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,\
                                                directory = '',\
                                                xcol='filename',ycol='class',\
                                                target_size=(222,222), color_mode='rgb',\
                                                batch_size=128, class_mode='categorical',\
                                                shuffle=False)

Found 3893 validated image filenames belonging to 5 classes.
Found 971 validated image filenames belonging to 5 classes.
Found 444 validated image filenames belonging to 5 classes.


In [24]:
# import tensorflowcv
# from tensorflowcv.model_provider import get_model
# from tensorflowcv.models.common import is_channels_first

from tensorflowcv.model_provider import get_model as tfcv_get_model
from tensorflowcv.model_provider import init_variables_from_state_dict as tfcv_init_variables_from_state_dict

# model = tensorflowcv.model_provider.get_model("SqueezeNet", use_pretrained=True)

model = tfcv_get_model("squeezenet_v1_1", pretrained=True, data_format="channels_first")

In [12]:
nb_classes = 5

# model = SqueezeNet(nb_classes, inputs=(3, 224, 224))

In [25]:
model.summary()

AttributeError: 'SqueezeNet' object has no attribute 'summary'

In [26]:
def categorical_tnr(y_true,y_pred):
    import keras.backend as K
    y_true = K.argmax(y_true)
    y_pred = K.argmax(y_pred)
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.cast(K.sum(neg_y_true * y_pred),'float32')
    tn = K.cast(K.sum(neg_y_true * neg_y_pred),'float32')
    tnr = tn / (tn + fp + K.epsilon())
    return tnr

def categorical_tpr(y_true,y_pred):
    import keras.backend as K
    y_true = K.argmax(y_true)
    y_pred = K.argmax(y_pred)
    neg_y_pred = 1 - y_pred
    fn = K.cast(K.sum(y_true * neg_y_pred),'float32')
    tp = K.cast(K.sum(y_true * y_pred),'float32')
    tpr = tp / (tp + fn + K.epsilon())
    return tpr

def categorical_tss(y_true,y_pred):
    import keras.backend as K
    tpr = categorical_tpr(y_true,y_pred)
    tnr = categorical_tnr(y_true,y_pred)
    tss = tpr + tnr - 1
    return tss

def save_con_mat(y_true, y_pred):
    import tensorflow as tf
    print((y_true.shape))
    print((y_pred.shape))
    con_matrix = tf.math.confusion_matrix(y_true, y_pred)
    # acc=np.diag(con_matrix).sum().astype(float)/con_matrix.sum()
    return con_matrix


In [27]:
from tensorflow.keras.optimizers import Adam
adam_opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam_opt,\
               metrics=['categorical_accuracy'])# save_con_mat]) #,categorical_tnr,categorical_tpr,categorical_tss])

AttributeError: 'SqueezeNet' object has no attribute 'compile'

In [16]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

filepath = 'BCS_CNN\\BCS_CNN.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_tss', verbose=1, save_best_only=False, mode='max')
early_stop = EarlyStopping(monitor='val_categorical_tss', min_delta=0.001, patience=5, verbose=1, mode='max')
#callbacks_list = [checkpoint, early_stop]
callbacks_list = [checkpoint]

step_size_train = np.ceil(train_generator.n/train_generator.batch_size)
print(step_size_train)
print('Batch size: ', train_generator.batch_size)
#step_size_train = 2373 # to help debug val accuracy issue
step_size_val = np.ceil(val_generator.n/val_generator.batch_size)
print(step_size_val)
#step_size_val = 303 # to help debug val accuracy issue

# the following assumes that 0 is the majority class
# class_weights = {0: 1.,
#                  1: (np.asarray(train_generator.classes)==0).sum()/(np.asarray(train_generator.classes)==1).sum()}
# the following taken from https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
#class_weights = {0: (1/(train_generator.classes==0).sum())*len(train_generator.classes)/2,
#                 1: (1/(train_generator.classes==1).sum())*len(train_generator.classes)/2}
# class_weights = {0: 1.,
#                 1: 6.}

from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_df['class']), y =train_df['class'])
class_weights = dict(enumerate(class_weights))

31.0
Batch size:  128
8.0


In [17]:
history = model.fit(train_generator, steps_per_epoch=step_size_train, epochs=10, verbose=1,\
                     callbacks=callbacks_list, validation_data=val_generator, validation_steps=step_size_val,\
                     validation_freq=1, class_weight=class_weights)

Epoch 1/10


NotFoundError:  No algorithm worked!
	 [[node model/conv1/Relu
 (defined at C:\Users\zacha\anaconda3\envs\ML_env\lib\site-packages\keras\backend.py:4867)
]] [Op:__inference_train_function_3248]

Errors may have originated from an input operation.
Input Source operations connected to node model/conv1/Relu:
In[0] model/conv1/BiasAdd (defined at C:\Users\zacha\anaconda3\envs\ML_env\lib\site-packages\keras\layers\convolutional.py:264)

Operation defined at: (most recent call last)
>>>   File "C:\Users\zacha\anaconda3\envs\ML_env\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\zacha\anaconda3\envs\ML_env\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\zacha\anaconda3\envs\ML_env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "C:\Users\zacha\anaconda3\envs\ML_env\lib\site-packages\traitlets\config\applica

In [ ]:
def normalize_Xtrain(X_train):
    mn = []
    mx = []
    Xn_train = np.zeros(np.shape(X_train))
    X_norm = np.zeros(np.shape(X_train))
    for i in range(len(X_train[0, ::])):
        mn.append(np.min(X_train[::, i]))
        Xn_train[::, i] = X_train[::, i] - mn[i]
    for i in range(len(X_train[0, ::])):
        mx.append(np.max(Xn_train[::, i]))
        if mx[i] == 0:
            X_norm[::, i] = 0
        if mx[i] != 0:
            X_norm[::, i] = Xn_train[::, i]/mx[i]
            
    return X_norm, mx, mn

In [ ]:
import sklearn.metrics

Y_test_hat=model.predict(test_generator)
y_test_hat=Y_test_hat.argmax(axis=-1)+2

print(y_test_hat[:200])


y_test = test_df['class']
y_test = [int(i) for i in y_test]
print(y_test[:200])

con_matrix = sklearn.metrics.confusion_matrix(y_test,y_test_hat)
acc=np.diag(con_matrix).sum().astype(float)/con_matrix.sum()
# acc = tensorflow.keras.metrics.Accuracy()
# acc.reset_state()
# acc.update_state(y_test, y_test_hat)


print('The accuracy of the network model2 is: ', acc)

min = np.min(con_matrix)
max = np.max(con_matrix)
temp_mat = con_matrix - min
temp_mat = con_matrix/max

plt.figure(figsize=(10,10))
plt.imshow(normalize_Xtrain(con_matrix)[0], cmap='gray')
plt.title('Confusion Matrix for Model2')



print(history.history.keys())
plt.figure(figsize=(10,10))
plt.plot(np.arange(0, len(history.history['categorical_accuracy'])), history.history['categorical_accuracy'])
plt.plot(np.arange(0, len(history.history['val_categorical_accuracy'])), history.history['val_categorical_accuracy'])
plt.legend(('Training Accuracy', 'Validation Accuracy'))